In [2]:
from transformers import LayoutLMv2Processor
from PIL import Image, ImageDraw, ImageFont
from datasets import load_dataset 
from transformers import LayoutLMv2ForTokenClassification

In [8]:
model = LayoutLMv2ForTokenClassification.from_pretrained("MarkusDressel/cord")


Downloading:   0%|          | 0.00/4.49k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/765M [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("mode/layoutLMv2_Cord")

In [ ]:
example = dataset["test"][1]

In [ ]:
processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased")

encoding = processor(image, return_offsets_mapping=True, return_tensors="pt")
offset_mapping = encoding.pop('offset_mapping')
print(encoding.keys())

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for k,v in encoding.items():
    encoding[k] = v.to(device)

In [ ]:


# load the fine-tuned model from the hub
model = LayoutLMv2ForTokenClassification.from_pretrained("mode/layoutLMv2_Cord")
model.to(device)

# forward pass
outputs = model(**encoding)

In [ ]:
id2label = model.config.id2label
label2id = model.config.label2id

In [ ]:
from PIL import ImageDraw

draw = ImageDraw.Draw(image)

font = ImageFont.load_default()

def iob_to_label(label):
    label = label[2:]
    if not label:
      return 'other'
    return label

label2color = {'question':'blue', 'answer':'green', 'header':'orange', 'other':'violet'}

for prediction, box in zip(true_predictions, true_boxes):
    predicted_label = iob_to_label(prediction).lower()
    draw.rectangle(box, outline=label2color[predicted_label])
    draw.text((box[0]+10, box[1]-10), text=predicted_label, fill=label2color[predicted_label], font=font)

image